In [30]:
import requests
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import time
import random


### safe request function
function that in the event of a page error retries the request until successful.

In [31]:
def safe_request_loop(url, headers):
    attempt = 0
    while True:
        try:
            response = requests.get(url, headers=headers)
            if response.status_code == 200:
                print('Nessun errore')
                return response
            else:
                print(f"Errore {response.status_code} - ritento...")
        except Exception as e:
            print(f"Eccezione nella richiesta: {e} - ritento...")

        # Attendi prima di riprovare
        wait_time = min(60, 2 ** attempt + random.uniform(0, 2))
        print(f"Aspetto {wait_time:.2f} secondi prima del tentativo successivo...")
        time.sleep(wait_time)
        attempt += 1

### function that extracts scorers for each match
We create a function that has the year and the day of the championship as parameters, these parameters will be inserted into the url that will lead back to the specific day of the championship, here all the scorers for each game of that day will be extracted. The ouput returned will be a list of dictionaries

In [48]:
def extract_goals(giornata, anno):
    match_data = []
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8",
        "Accept-Language": "it-IT,it;q=0.9,en-US;q=0.8,en;q=0.7",
        "Referer": "https://www.google.com/"
        }
    url = f'https://www.transfermarkt.it/serie-a/spieltag/wettbewerb/IT1/plus/?saison_id={anno}&spieltag={giornata}'
    response = safe_request_loop(url,headers)#requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')

    section = soup.find('div', class_='large-8 columns')

    boxes = section.find_all("div", class_ = "box")

    for box in boxes:
        # Salta box che non contengono risultati
        if not box.find("span", class_="matchresult"):
            continue

        try:
            # Squadre e risultato
            home_team_box = box.find("td", class_ = 'rechts hauptlink no-border-rechts hide-for-small spieltagsansicht-vereinsname')
            #home_team = home_team_box.find("a", href = True).text.strip()

            if home_team_box:
                for a in home_team_box.find_all("a", href=True):
                    if a.get("title") != "Discutere della giornata" and a.text.strip():
                        home_team = a.text.strip()
                        break
            away_team_box = box.find("td", class_ = 'hauptlink no-border-links no-border-rechts hide-for-small spieltagsansicht-vereinsname')
            away_team = away_team_box.find("a", href = True).text.strip()

            result = box.find("span", class_="matchresult").text.strip()

            #match_data.append({
            #    "Squadra Casa": home_team,
            #    "Squadra Trasferta": away_team,
            #    "Risultato Finale": result
            #})

            # Estrai tutti i goal
            trs = box.find_all("tr", class_="spieltagsansicht-aktionen")

            #tds = box.find_all('td', class_='spieltagsansicht')

            for tr in trs:
                # Estrai il nome del giocatore
                player_span = tr.find('span', class_='hide-for-small')
                if player_span and player_span.find('a'):
                    player_name = player_span.find('a').text.strip()
                else:
                    continue  # Salta se non trova il nome
                
                # Estrai il minuto del gol dal titolo dell'icona
                goal_icon = tr.find('span', class_='icon-tor-formation') or tr.find('span', class_='icon-elfmeter-formation')
                if goal_icon and 'title' in goal_icon.attrs:
                    minute_info = goal_icon['title']
                    # Estrai il numero dal titolo, es. '2.min: gol'
                    minute = minute_info.split('.')[0]
                else:
                    continue  # Salta se non trova l'icona del gol

                goal_result_td = tr.find("td", class_="zentriert hauptlink")
                goal_result = goal_result_td.get_text(strip=True) if goal_result_td else None
    

                match_data.append({
                    "season": anno,
                    "matchweek":giornata,
                    "home_team": home_team,
                    "away_team": away_team,
                    "goal_result": goal_result,
                    "scorer": player_name,
                    "minute": minute
                    })
                print(f"Giocatore: {player_name}, Minuto: {minute}, Squadra casa: {home_team}, Squadra trasferta: {away_team}, Risultato al goal: {goal_result}")

        except Exception as e:
            print(f"Errore durante l'elaborazione di una partita: {e}")

    return match_data





For loop that cycles over year and day and extracts for each match the scorers

In [49]:
anni = [2021,2022,2023,2024]
goals = []
for anno in anni:
    for giornata in range(1,39):
        goals.extend(extract_goals(giornata, anno))

df_goals = pd.DataFrame(goals)

Nessun errore
Giocatore: Giacomo Raspadori, Minuto: 32, Squadra casa: Verona, Squadra trasferta: Sassuolo, Risultato al goal: 0:1
Giocatore: Filip Djuricic, Minuto: 51, Squadra casa: Verona, Squadra trasferta: Sassuolo, Risultato al goal: 0:2
Giocatore: Mattia Zaccagni, Minuto: 71, Squadra casa: Verona, Squadra trasferta: Sassuolo, Risultato al goal: 1:2
Giocatore: Hamed Junior Traoré, Minuto: 77, Squadra casa: Verona, Squadra trasferta: Sassuolo, Risultato al goal: 1:3
Giocatore: Mattia Zaccagni, Minuto: 90, Squadra casa: Verona, Squadra trasferta: Sassuolo, Risultato al goal: 2:3
Giocatore: Milan Škriniar, Minuto: 6, Squadra casa: Inter, Squadra trasferta: Genoa, Risultato al goal: 1:0
Giocatore: Hakan Çalhanoğlu, Minuto: 14, Squadra casa: Inter, Squadra trasferta: Genoa, Risultato al goal: 2:0
Giocatore: Arturo Vidal, Minuto: 74, Squadra casa: Inter, Squadra trasferta: Genoa, Risultato al goal: 3:0
Giocatore: Edin Džeko, Minuto: 87, Squadra casa: Inter, Squadra trasferta: Genoa, Ris

In [51]:
df_goals.to_csv("dataset/matches_goal.csv", index= False, sep = ';')